In [32]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.layers import Dense
from numpy.random import default_rng
from sklearn.model_selection import train_test_split
import math

In [33]:
old_full_data = pd.read_csv('data/full_data_10HZ.csv', sep=",", header=None, names=["e_t_x","e_t_y","e_t_z","e_r_x","e_r_y","e_r_z","e_r_w","d_t_x","d_t_y","d_t_z","d_r_x","d_r_y","d_r_z","d_r_w", "c_t_x","c_t_y","c_t_z","c_r_x","c_r_y","c_r_z", "c_r_w",  "speed"])
old_full_data
old_full_data.tail()


,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
41995,-0.156,0.652,0.331,-0.035,0.666,0.745,0.019,0.097,0.615,0.172,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
41996,-0.161,0.676,0.464,0.067,-0.750,-0.658,0.010,0.097,0.613,0.170,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
41997,-0.166,0.682,0.525,-0.085,0.753,0.652,0.006,0.096,0.611,0.168,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
41998,-0.156,0.668,0.469,0.032,-0.745,-0.666,0.031,0.095,0.610,0.166,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast
41999,-0.149,0.645,0.383,-0.002,-0.726,-0.685,0.062,0.096,0.611,0.165,...,0.737,0.010,0.049,2.274,0.271,0.019,0.017,-0.027,0.999,fast


In [34]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df


In [35]:

 
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

In [36]:

def get_quaternion_from_euler(roll, pitch, yaw):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
    return qx, qy, qz, qw

In [37]:
data_stop_0 = old_full_data.loc[old_full_data['speed']=="stopped"][1:].reset_index(drop=True)
data_stop_0

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.331,0.652,0.209,0.004,0.677,0.736,0.010,0.313,0.643,0.216,...,0.752,0.071,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,-0.331,0.652,0.209,0.004,0.677,0.735,0.010,0.313,0.643,0.216,...,0.753,0.071,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,-0.331,0.651,0.209,0.004,0.677,0.735,0.010,0.313,0.643,0.217,...,0.752,0.071,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,-0.331,0.652,0.209,0.004,0.677,0.735,0.010,0.313,0.643,0.216,...,0.752,0.071,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,-0.331,0.652,0.209,0.004,0.678,0.735,0.010,0.313,0.643,0.216,...,0.752,0.071,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,-0.125,0.614,0.176,-0.014,-0.688,-0.723,0.058,0.133,0.610,0.174,...,-0.728,0.020,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,-0.125,0.613,0.176,-0.013,-0.688,-0.723,0.058,0.133,0.610,0.174,...,-0.727,0.019,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,-0.125,0.613,0.176,-0.013,-0.688,-0.723,0.058,0.133,0.609,0.174,...,-0.727,0.019,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,-0.125,0.613,0.176,-0.013,-0.688,-0.723,0.058,0.132,0.610,0.175,...,-0.728,0.019,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [38]:
data_stop_90_left_foot_without_negativation = swap_columns(data_stop_0, 'e_t_x', 'e_t_z')
data_stop_90_left_foot_without_negativation["e_t_z"] = data_stop_90_left_foot_without_negativation["e_t_z"]*-1
data_stop_90_left_foot=data_stop_90_left_foot_without_negativation
data_stop_90_left_right_foot_without_negativation = swap_columns(data_stop_90_left_foot, 'd_t_x', 'd_t_z')
data_stop_90_left_right_foot_without_negativation["d_t_z"] = data_stop_90_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_90=data_stop_90_left_right_foot_without_negativation

data_stop_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_90.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_90.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_90.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_90

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.209,0.652,-0.331,0.004,0.677,0.736,0.010,-0.216,0.643,0.313,...,0.752,0.071,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,-0.209,0.652,-0.331,0.004,0.677,0.735,0.010,-0.216,0.643,0.313,...,0.753,0.071,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,-0.209,0.651,-0.331,0.004,0.677,0.735,0.010,-0.217,0.643,0.313,...,0.752,0.071,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,-0.209,0.652,-0.331,0.004,0.677,0.735,0.010,-0.216,0.643,0.313,...,0.752,0.071,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,-0.209,0.652,-0.331,0.004,0.678,0.735,0.010,-0.216,0.643,0.313,...,0.752,0.071,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,-0.176,0.614,-0.125,-0.014,-0.688,-0.723,0.058,-0.174,0.610,0.133,...,-0.728,0.020,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,-0.176,0.613,-0.125,-0.013,-0.688,-0.723,0.058,-0.174,0.610,0.133,...,-0.727,0.019,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,-0.176,0.613,-0.125,-0.013,-0.688,-0.723,0.058,-0.174,0.609,0.133,...,-0.727,0.019,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,-0.176,0.613,-0.125,-0.013,-0.688,-0.723,0.058,-0.175,0.610,0.132,...,-0.728,0.019,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [39]:
data_stop_180_left_foot_without_negativation = swap_columns(data_stop_90, 'e_t_x', 'e_t_z')
data_stop_180_left_foot_without_negativation["e_t_z"] = data_stop_180_left_foot_without_negativation["e_t_z"]*-1
data_stop_180_left_foot=data_stop_180_left_foot_without_negativation
data_stop_180_left_right_foot_without_negativation = swap_columns(data_stop_180_left_foot, 'd_t_x', 'd_t_z')
data_stop_180_left_right_foot_without_negativation["d_t_z"] = data_stop_180_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_180=data_stop_180_left_right_foot_without_negativation

data_stop_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_180.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_180.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_180.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_180

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.331,0.652,-0.209,0.004,0.677,0.736,0.010,-0.313,0.643,-0.216,...,0.752,0.071,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,0.331,0.652,-0.209,0.004,0.677,0.735,0.010,-0.313,0.643,-0.216,...,0.753,0.071,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,0.331,0.651,-0.209,0.004,0.677,0.735,0.010,-0.313,0.643,-0.217,...,0.752,0.071,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,0.331,0.652,-0.209,0.004,0.677,0.735,0.010,-0.313,0.643,-0.216,...,0.752,0.071,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,0.331,0.652,-0.209,0.004,0.678,0.735,0.010,-0.313,0.643,-0.216,...,0.752,0.071,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,0.125,0.614,-0.176,-0.014,-0.688,-0.723,0.058,-0.133,0.610,-0.174,...,-0.728,0.020,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,0.125,0.613,-0.176,-0.013,-0.688,-0.723,0.058,-0.133,0.610,-0.174,...,-0.727,0.019,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,0.125,0.613,-0.176,-0.013,-0.688,-0.723,0.058,-0.133,0.609,-0.174,...,-0.727,0.019,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,0.125,0.613,-0.176,-0.013,-0.688,-0.723,0.058,-0.132,0.610,-0.175,...,-0.728,0.019,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [40]:
data_stop_270_left_foot_without_negativation = swap_columns(data_stop_180, 'e_t_x', 'e_t_z')
data_stop_270_left_foot_without_negativation["e_t_z"] = data_stop_270_left_foot_without_negativation["e_t_z"]*-1
data_stop_270_left_foot=data_stop_270_left_foot_without_negativation
data_stop_270_left_right_foot_without_negativation = swap_columns(data_stop_270_left_foot, 'd_t_x', 'd_t_z')
data_stop_270_left_right_foot_without_negativation["d_t_z"] = data_stop_270_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_270=data_stop_270_left_right_foot_without_negativation

data_stop_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_270.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_stop_270.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_stop_270.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_stop_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.209,0.652,0.331,0.004,0.677,0.736,0.010,0.216,0.643,-0.313,...,0.752,0.071,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,0.209,0.652,0.331,0.004,0.677,0.735,0.010,0.216,0.643,-0.313,...,0.753,0.071,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,0.209,0.651,0.331,0.004,0.677,0.735,0.010,0.217,0.643,-0.313,...,0.752,0.071,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,0.209,0.652,0.331,0.004,0.677,0.735,0.010,0.216,0.643,-0.313,...,0.752,0.071,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,0.209,0.652,0.331,0.004,0.678,0.735,0.010,0.216,0.643,-0.313,...,0.752,0.071,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,0.176,0.614,0.125,-0.014,-0.688,-0.723,0.058,0.174,0.610,-0.133,...,-0.728,0.020,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,0.176,0.613,0.125,-0.013,-0.688,-0.723,0.058,0.174,0.610,-0.133,...,-0.727,0.019,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,0.176,0.613,0.125,-0.013,-0.688,-0.723,0.058,0.174,0.609,-0.133,...,-0.727,0.019,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,0.176,0.613,0.125,-0.013,-0.688,-0.723,0.058,0.175,0.610,-0.132,...,-0.728,0.019,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [10]:
data_stop_90.loc[:,["d_r_x","d_r_y","d_r_z","d_r_w" ]]

,d_r_x,d_r_y,d_r_z,d_r_w
0,0.221,0.616,0.752,0.071
1,0.221,0.616,0.753,0.071
2,0.221,0.616,0.752,0.071
3,0.221,0.616,0.752,0.071
4,0.221,0.616,0.752,0.071
...,...,...,...,...
17994,-0.111,-0.677,-0.728,0.020
17995,-0.111,-0.677,-0.727,0.019
17996,-0.111,-0.677,-0.727,0.019
17997,-0.111,-0.677,-0.728,0.019


In [11]:
for i in data_stop_90:
    print(i)

e_t_x
e_t_y
e_t_z
e_r_x
e_r_y
e_r_z
e_r_w
d_t_x
d_t_y
d_t_z
d_r_x
d_r_y
d_r_z
d_r_w
c_t_x
c_t_y
c_t_z
c_r_x
c_r_y
c_r_z
c_r_w
speed


In [12]:
l_e_r_x_quat_data_stop_90=[]
l_e_r_y_quat_data_stop_90=[]
l_e_r_z_quat_data_stop_90=[]
l_e_r_w_quat_data_stop_90=[]
l_d_r_x_quat_data_stop_90=[]
l_d_r_y_quat_data_stop_90=[]
l_d_r_z_quat_data_stop_90=[]
l_d_r_w_quat_data_stop_90=[]
for index_90, row_90 in data_stop_90.iterrows():
    e_r_x_euler_data_stop_90,e_r_y_euler_data_stop_90,e_r_z_euler_data_stop_90=euler_from_quaternion(row_90[3],row_90[4],row_90[5],row_90[6])
    d_r_x_euler_data_stop_90,d_r_y_euler_data_stop_90,d_r_z_euler_data_stop_90=euler_from_quaternion(row_90[10],row_90[11],row_90[12],row_90[13])
    
    e_r_z_euler_data_stop_90=e_r_z_euler_data_stop_90+np.pi/2
    d_r_z_euler_data_stop_90=d_r_z_euler_data_stop_90+np.pi/2
    
    e_r_x_data_stop_90, e_r_y_data_stop_90, e_r_z_data_stop_90,e_r_w_data_stop_90=get_quaternion_from_euler(e_r_x_euler_data_stop_90, e_r_y_euler_data_stop_90,e_r_z_euler_data_stop_90)
    d_r_x_data_stop_90, d_r_y_data_stop_90, d_r_z_data_stop_90,d_r_w_data_stop_90=get_quaternion_from_euler(d_r_x_euler_data_stop_90, d_r_y_euler_data_stop_90,d_r_z_euler_data_stop_90)
    
    l_e_r_x_quat_data_stop_90.append(e_r_x_data_stop_90)
    l_e_r_y_quat_data_stop_90.append(e_r_y_data_stop_90)
    l_e_r_z_quat_data_stop_90.append(e_r_z_data_stop_90)
    l_e_r_w_quat_data_stop_90.append(e_r_w_data_stop_90)
    l_d_r_x_quat_data_stop_90.append(d_r_x_data_stop_90)
    l_d_r_y_quat_data_stop_90.append(d_r_y_data_stop_90)
    l_d_r_z_quat_data_stop_90.append(d_r_z_data_stop_90)
    l_d_r_w_quat_data_stop_90.append(d_r_w_data_stop_90)
    
data_stop_90['e_r_x']=l_e_r_x_quat_data_stop_90
data_stop_90['e_r_y']=l_e_r_y_quat_data_stop_90
data_stop_90['e_r_z']=l_e_r_z_quat_data_stop_90
data_stop_90['e_r_w']=l_e_r_w_quat_data_stop_90
data_stop_90['d_r_x']=l_d_r_x_quat_data_stop_90
data_stop_90['d_r_y']=l_d_r_y_quat_data_stop_90
data_stop_90['d_r_z']=l_d_r_z_quat_data_stop_90
data_stop_90['d_r_w']=l_d_r_w_quat_data_stop_90


In [13]:
data_stop_90

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.209,0.652,-0.331,-0.475886,0.481542,0.527430,-0.513290,-0.216,0.643,0.313,...,0.582813,-0.481776,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,-0.209,0.652,-0.331,-0.475850,0.481519,0.527469,-0.513305,-0.216,0.643,0.313,...,0.582397,-0.482176,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,-0.209,0.651,-0.331,-0.475850,0.481519,0.527469,-0.513305,-0.217,0.643,0.313,...,0.582813,-0.481776,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,-0.209,0.652,-0.331,-0.475850,0.481519,0.527469,-0.513305,-0.216,0.643,0.313,...,0.582813,-0.481776,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,-0.209,0.652,-0.331,-0.476591,0.482247,0.526782,-0.512640,-0.216,0.643,0.313,...,0.582813,-0.481776,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,-0.176,0.614,-0.125,0.476591,-0.496378,-0.470407,0.552437,-0.174,0.610,0.133,...,0.499996,-0.528511,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,-0.176,0.613,-0.125,0.477299,-0.495670,-0.470420,0.552451,-0.174,0.610,0.133,...,0.500915,-0.527681,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,-0.176,0.613,-0.125,0.477299,-0.495670,-0.470420,0.552451,-0.174,0.609,0.133,...,0.500915,-0.527681,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,-0.176,0.613,-0.125,0.477299,-0.495670,-0.470420,0.552451,-0.175,0.610,0.132,...,0.500727,-0.527820,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [14]:
data_stop_90.loc[:,["d_r_x","d_r_y","d_r_z","d_r_w" ]]

,d_r_x,d_r_y,d_r_z,d_r_w
0,-0.279144,0.591861,0.582813,-0.481776
1,-0.279356,0.591845,0.582397,-0.482176
2,-0.279144,0.591861,0.582813,-0.481776
3,-0.279144,0.591861,0.582813,-0.481776
4,-0.279144,0.591861,0.582813,-0.481776
...,...,...,...,...
17994,-0.400271,0.557192,0.499996,-0.528511
17995,-0.400201,0.557204,0.500915,-0.527681
17996,-0.400201,0.557204,0.500915,-0.527681
17997,-0.400270,0.557192,0.500727,-0.527820


In [15]:
l_e_r_x_quat_data_stop_180=[]
l_e_r_y_quat_data_stop_180=[]
l_e_r_z_quat_data_stop_180=[]
l_e_r_w_quat_data_stop_180=[]
l_d_r_x_quat_data_stop_180=[]
l_d_r_y_quat_data_stop_180=[]
l_d_r_z_quat_data_stop_180=[]
l_d_r_w_quat_data_stop_180=[]
for index_180, row_180 in data_stop_180.iterrows():
    e_r_x_euler_data_stop_180,e_r_y_euler_data_stop_180,e_r_z_euler_data_stop_180=euler_from_quaternion(row_180[3],row_180[4],row_180[5],row_180[6])
    d_r_x_euler_data_stop_180,d_r_y_euler_data_stop_180,d_r_z_euler_data_stop_180=euler_from_quaternion(row_180[10],row_180[11],row_180[12],row_180[13])
    
    e_r_z_euler_data_stop_180=e_r_z_euler_data_stop_180+np.pi
    d_r_z_euler_data_stop_180=d_r_z_euler_data_stop_180+np.pi
    
    e_r_x_data_stop_180, e_r_y_data_stop_180, e_r_z_data_stop_180,e_r_w_data_stop_180=get_quaternion_from_euler(e_r_x_euler_data_stop_180, e_r_y_euler_data_stop_180,e_r_z_euler_data_stop_180)
    d_r_x_data_stop_180, d_r_y_data_stop_180, d_r_z_data_stop_180,d_r_w_data_stop_180=get_quaternion_from_euler(d_r_x_euler_data_stop_180, d_r_y_euler_data_stop_180,d_r_z_euler_data_stop_180)
    
    l_e_r_x_quat_data_stop_180.append(e_r_x_data_stop_180)
    l_e_r_y_quat_data_stop_180.append(e_r_y_data_stop_180)
    l_e_r_z_quat_data_stop_180.append(e_r_z_data_stop_180)
    l_e_r_w_quat_data_stop_180.append(e_r_w_data_stop_180)
    l_d_r_x_quat_data_stop_180.append(d_r_x_data_stop_180)
    l_d_r_y_quat_data_stop_180.append(d_r_y_data_stop_180)
    l_d_r_z_quat_data_stop_180.append(d_r_z_data_stop_180)
    l_d_r_w_quat_data_stop_180.append(d_r_w_data_stop_180)
    
data_stop_180['e_r_x']=l_e_r_x_quat_data_stop_180
data_stop_180['e_r_y']=l_e_r_y_quat_data_stop_180
data_stop_180['e_r_z']=l_e_r_z_quat_data_stop_180
data_stop_180['e_r_w']=l_e_r_w_quat_data_stop_180
data_stop_180['d_r_x']=l_d_r_x_quat_data_stop_180
data_stop_180['d_r_y']=l_d_r_y_quat_data_stop_180
data_stop_180['d_r_z']=l_d_r_z_quat_data_stop_180
data_stop_180['d_r_w']=l_d_r_w_quat_data_stop_180


In [16]:
data_stop_180

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.331,0.652,-0.209,-0.677004,0.003999,0.009998,-0.735901,-0.313,0.643,-0.216,...,0.071444,-0.752778,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,0.331,0.652,-0.209,-0.676962,0.004009,0.010016,-0.735939,-0.313,0.643,-0.216,...,0.070867,-0.752767,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,0.331,0.651,-0.209,-0.676962,0.004009,0.010016,-0.735939,-0.313,0.643,-0.217,...,0.071444,-0.752778,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,0.331,0.652,-0.209,-0.676962,0.004009,0.010016,-0.735939,-0.313,0.643,-0.216,...,0.071444,-0.752778,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,0.331,0.652,-0.209,-0.678001,0.004000,0.010000,-0.734982,-0.313,0.643,-0.216,...,0.071444,-0.752778,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,0.125,0.614,-0.176,0.687993,-0.013992,0.058004,0.723260,-0.133,0.610,-0.174,...,-0.020163,-0.727265,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
17995,0.125,0.613,-0.176,0.687993,-0.012991,0.058005,0.723279,-0.133,0.610,-0.174,...,-0.018927,-0.727327,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
17996,0.125,0.613,-0.176,0.687993,-0.012991,0.058005,0.723279,-0.133,0.609,-0.174,...,-0.018927,-0.727327,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
17997,0.125,0.613,-0.176,0.687993,-0.012991,0.058005,0.723279,-0.132,0.610,-0.175,...,-0.019157,-0.727293,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [17]:
l_e_r_x_quat_data_stop_270=[]
l_e_r_y_quat_data_stop_270=[]
l_e_r_z_quat_data_stop_270=[]
l_e_r_w_quat_data_stop_270=[]
l_d_r_x_quat_data_stop_270=[]
l_d_r_y_quat_data_stop_270=[]
l_d_r_z_quat_data_stop_270=[]
l_d_r_w_quat_data_stop_270=[]
for index_270, row_270 in data_stop_270.iterrows():
    e_r_x_euler_data_stop_270,e_r_y_euler_data_stop_270,e_r_z_euler_data_stop_270=euler_from_quaternion(row_270[3],row_270[4],row_270[5],row_270[6])
    d_r_x_euler_data_stop_270,d_r_y_euler_data_stop_270,d_r_z_euler_data_stop_270=euler_from_quaternion(row_270[10],row_270[11],row_270[12],row_270[13])
    
    e_r_z_euler_data_stop_270=e_r_z_euler_data_stop_270+np.pi*3/4
    d_r_z_euler_data_stop_270=d_r_z_euler_data_stop_270+np.pi*3/4
    
    e_r_x_data_stop_270, e_r_y_data_stop_270, e_r_z_data_stop_270,e_r_w_data_stop_270=get_quaternion_from_euler(e_r_x_euler_data_stop_270, e_r_y_euler_data_stop_270,e_r_z_euler_data_stop_270)
    d_r_x_data_stop_270, d_r_y_data_stop_270, d_r_z_data_stop_270,d_r_w_data_stop_270=get_quaternion_from_euler(d_r_x_euler_data_stop_270, d_r_y_euler_data_stop_270,d_r_z_euler_data_stop_270)
    
    l_e_r_x_quat_data_stop_270.append(e_r_x_data_stop_270)
    l_e_r_y_quat_data_stop_270.append(e_r_y_data_stop_270)
    l_e_r_z_quat_data_stop_270.append(e_r_z_data_stop_270)
    l_e_r_w_quat_data_stop_270.append(e_r_w_data_stop_270)
    l_d_r_x_quat_data_stop_270.append(d_r_x_data_stop_270)
    l_d_r_y_quat_data_stop_270.append(d_r_y_data_stop_270)
    l_d_r_z_quat_data_stop_270.append(d_r_z_data_stop_270)
    l_d_r_w_quat_data_stop_270.append(d_r_w_data_stop_270)
    
data_stop_270['e_r_x']=l_e_r_x_quat_data_stop_270
data_stop_270['e_r_y']=l_e_r_y_quat_data_stop_270
data_stop_270['e_r_z']=l_e_r_z_quat_data_stop_270
data_stop_270['e_r_w']=l_e_r_w_quat_data_stop_270
data_stop_270['d_r_x']=l_d_r_x_quat_data_stop_270
data_stop_270['d_r_y']=l_d_r_y_quat_data_stop_270
data_stop_270['d_r_z']=l_d_r_z_quat_data_stop_270
data_stop_270['d_r_w']=l_d_r_w_quat_data_stop_270



In [18]:
data_stop_0_90 = data_stop_0.append(data_stop_90, ignore_index=True)
data_stop_0_90_180 = data_stop_0_90.append(data_stop_180, ignore_index=True)
data_stop = data_stop_0_90_180.append(data_stop_270, ignore_index=True)
data_stop

/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_4267/2689906575.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop_0_90 = data_stop_0.append(data_stop_90, ignore_index=True)
/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_4267/2689906575.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop_0_90_180 = data_stop_0_90.append(data_stop_180, ignore_index=True)
/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_4267/2689906575.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop = data_stop_0_90_180.append(data_stop_270, ignore_index=True)


,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.331,0.652,0.209,0.004000,0.677000,0.736000,0.010000,0.313,0.643,0.216,...,0.752000,0.071000,-0.028,2.288,0.179,-0.100,-0.025,-0.045,0.994,stopped
1,-0.331,0.652,0.209,0.004000,0.677000,0.735000,0.010000,0.313,0.643,0.216,...,0.753000,0.071000,-0.021,2.288,0.180,-0.106,-0.014,-0.054,0.993,stopped
2,-0.331,0.651,0.209,0.004000,0.677000,0.735000,0.010000,0.313,0.643,0.217,...,0.752000,0.071000,-0.013,2.288,0.181,-0.104,-0.001,-0.062,0.993,stopped
3,-0.331,0.652,0.209,0.004000,0.677000,0.735000,0.010000,0.313,0.643,0.216,...,0.752000,0.071000,-0.004,2.287,0.181,-0.106,0.014,-0.069,0.992,stopped
4,-0.331,0.652,0.209,0.004000,0.678000,0.735000,0.010000,0.313,0.643,0.216,...,0.752000,0.071000,0.002,2.287,0.180,-0.105,0.023,-0.073,0.992,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71991,0.176,0.614,0.125,0.630269,-0.276210,-0.223191,0.690402,0.174,0.610,-0.133,...,0.259684,-0.679621,-0.002,2.290,0.188,0.016,-0.045,-0.003,0.999,stopped
71992,0.176,0.613,0.125,0.630651,-0.275285,-0.223197,0.690420,0.174,0.610,-0.133,...,0.260850,-0.679206,-0.009,2.289,0.189,0.018,-0.071,0.004,0.997,stopped
71993,0.176,0.613,0.125,0.630651,-0.275285,-0.223197,0.690420,0.174,0.609,-0.133,...,0.260850,-0.679206,-0.014,2.289,0.185,0.024,-0.083,0.010,0.996,stopped
71994,0.176,0.613,0.125,0.630651,-0.275285,-0.223197,0.690420,0.175,0.610,-0.132,...,0.260624,-0.679262,-0.022,2.293,0.180,0.010,-0.094,0.026,0.995,stopped


In [19]:
data_0_slow= old_full_data.loc[old_full_data['speed']=="slow"][1:].reset_index(drop=True)
data_0_fast=old_full_data.loc[old_full_data['speed']=="fast"][1:].reset_index(drop=True)

In [20]:
data_0 = data_0_slow.append(data_0_fast, ignore_index=True)
data_0

/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_4267/1875441389.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_0 = data_0_slow.append(data_0_fast, ignore_index=True)


,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.162,0.622,0.120,0.006,-0.654,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.014,0.193,0.623,0.137,...,0.769,0.123,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,-0.156,0.652,0.331,-0.035,0.666,0.745,0.019,0.097,0.615,0.172,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,-0.161,0.676,0.464,0.067,-0.750,-0.658,0.010,0.097,0.613,0.170,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,-0.166,0.682,0.525,-0.085,0.753,0.652,0.006,0.096,0.611,0.168,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,-0.156,0.668,0.469,0.032,-0.745,-0.666,0.031,0.095,0.610,0.166,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast


In [21]:
data_90_left_foot_without_negativation = swap_columns(data_0, 'e_t_x', 'e_t_z')
data_90_left_foot_without_negativation["e_t_z"] = data_90_left_foot_without_negativation["e_t_z"]*-1
data_90_left_foot=data_90_left_foot_without_negativation
data_90_left_right_foot_without_negativation = swap_columns(data_90_left_foot, 'd_t_x', 'd_t_z')
data_90_left_right_foot_without_negativation["d_t_z"] = data_90_left_right_foot_without_negativation["d_t_z"]*-1
data_90=data_90_left_right_foot_without_negativation

data_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_90.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_90.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_90.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_90

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.120,0.622,-0.162,0.006,-0.654,-0.756,0.015,-0.137,0.623,0.193,...,0.769,0.121,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,-0.120,0.622,-0.162,0.006,-0.655,-0.756,0.015,-0.137,0.623,0.193,...,0.769,0.121,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,-0.120,0.622,-0.162,0.006,-0.655,-0.756,0.015,-0.137,0.623,0.193,...,0.769,0.121,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,-0.120,0.622,-0.162,0.006,-0.655,-0.756,0.015,-0.137,0.623,0.193,...,0.769,0.121,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,-0.120,0.622,-0.162,0.006,-0.655,-0.756,0.014,-0.137,0.623,0.193,...,0.769,0.123,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,-0.331,0.652,-0.156,-0.035,0.666,0.745,0.019,-0.172,0.615,0.097,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,-0.464,0.676,-0.161,0.067,-0.750,-0.658,0.010,-0.170,0.613,0.097,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,-0.525,0.682,-0.166,-0.085,0.753,0.652,0.006,-0.168,0.611,0.096,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,-0.469,0.668,-0.156,0.032,-0.745,-0.666,0.031,-0.166,0.610,0.095,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast


In [22]:
data_180_left_foot_without_negativation = swap_columns(data_90, 'e_t_x', 'e_t_z')
data_180_left_foot_without_negativation["e_t_z"] = data_180_left_foot_without_negativation["e_t_z"]*-1
data_180_left_foot=data_180_left_foot_without_negativation
data_180_left_right_foot_without_negativation = swap_columns(data_180_left_foot, 'd_t_x', 'd_t_z')
data_180_left_right_foot_without_negativation["d_t_z"] = data_180_left_right_foot_without_negativation["d_t_z"]*-1
data_180=data_180_left_right_foot_without_negativation

data_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_180.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_180.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_180.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_180

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.162,0.622,-0.120,0.006,-0.654,-0.756,0.015,-0.193,0.623,-0.137,...,0.769,0.121,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,0.162,0.622,-0.120,0.006,-0.655,-0.756,0.015,-0.193,0.623,-0.137,...,0.769,0.121,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,0.162,0.622,-0.120,0.006,-0.655,-0.756,0.015,-0.193,0.623,-0.137,...,0.769,0.121,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,0.162,0.622,-0.120,0.006,-0.655,-0.756,0.015,-0.193,0.623,-0.137,...,0.769,0.121,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,0.162,0.622,-0.120,0.006,-0.655,-0.756,0.014,-0.193,0.623,-0.137,...,0.769,0.123,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.156,0.652,-0.331,-0.035,0.666,0.745,0.019,-0.097,0.615,-0.172,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,0.161,0.676,-0.464,0.067,-0.750,-0.658,0.010,-0.097,0.613,-0.170,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,0.166,0.682,-0.525,-0.085,0.753,0.652,0.006,-0.096,0.611,-0.168,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,0.156,0.668,-0.469,0.032,-0.745,-0.666,0.031,-0.095,0.610,-0.166,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast


In [23]:
data_270_left_foot_without_negativation = swap_columns(data_180, 'e_t_x', 'e_t_z')
data_270_left_foot_without_negativation["e_t_z"] = data_270_left_foot_without_negativation["e_t_z"]*-1
data_270_left_foot=data_270_left_foot_without_negativation
data_270_left_right_foot_without_negativation = swap_columns(data_270_left_foot, 'd_t_x', 'd_t_z')
data_270_left_right_foot_without_negativation["d_t_z"] = data_270_left_right_foot_without_negativation["d_t_z"]*-1
data_270=data_270_left_right_foot_without_negativation

data_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_270.rename(columns={'d_t_z':'d_t_x2'}, inplace=True)
data_270.rename(columns={'d_t_x':'d_t_z'}, inplace=True)
data_270.rename(columns={'d_t_x2':'d_t_x'}, inplace=True)
data_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.120,0.622,0.162,0.006,-0.654,-0.756,0.015,0.137,0.623,-0.193,...,0.769,0.121,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,0.120,0.622,0.162,0.006,-0.655,-0.756,0.015,0.137,0.623,-0.193,...,0.769,0.121,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,0.120,0.622,0.162,0.006,-0.655,-0.756,0.015,0.137,0.623,-0.193,...,0.769,0.121,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,0.120,0.622,0.162,0.006,-0.655,-0.756,0.015,0.137,0.623,-0.193,...,0.769,0.121,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,0.120,0.622,0.162,0.006,-0.655,-0.756,0.014,0.137,0.623,-0.193,...,0.769,0.123,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.331,0.652,0.156,-0.035,0.666,0.745,0.019,0.172,0.615,-0.097,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,0.464,0.676,0.161,0.067,-0.750,-0.658,0.010,0.170,0.613,-0.097,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,0.525,0.682,0.166,-0.085,0.753,0.652,0.006,0.168,0.611,-0.096,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,0.469,0.668,0.156,0.032,-0.745,-0.666,0.031,0.166,0.610,-0.095,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast


In [24]:
l_e_r_x_quat_data_90=[]
l_e_r_y_quat_data_90=[]
l_e_r_z_quat_data_90=[]
l_e_r_w_quat_data_90=[]
l_d_r_x_quat_data_90=[]
l_d_r_y_quat_data_90=[]
l_d_r_z_quat_data_90=[]
l_d_r_w_quat_data_90=[]
for index_m_90, row_m_90 in data_90.iterrows():
    e_r_x_euler_data_90,e_r_y_euler_data_90,e_r_z_euler_data_90=euler_from_quaternion(row_m_90[3],row_m_90[4],row_m_90[5],row_m_90[6])
    d_r_x_euler_data_90,d_r_y_euler_data_90,d_r_z_euler_data_90=euler_from_quaternion(row_m_90[10],row_m_90[11],row_m_90[12],row_m_90[13])
    
    e_r_z_euler_data_90=e_r_z_euler_data_90+np.pi/2
    d_r_z_euler_data_90=d_r_z_euler_data_90+np.pi/2
    
    e_r_x_data_90, e_r_y_data_90, e_r_z_data_90,e_r_w_data_90=get_quaternion_from_euler(e_r_x_euler_data_90, e_r_y_euler_data_90,e_r_z_euler_data_90)
    d_r_x_data_90, d_r_y_data_90, d_r_z_data_90,d_r_w_data_90=get_quaternion_from_euler(d_r_x_euler_data_90, d_r_y_euler_data_90,d_r_z_euler_data_90)
    
    l_e_r_x_quat_data_90.append(e_r_x_data_90)
    l_e_r_y_quat_data_90.append(e_r_y_data_90)
    l_e_r_z_quat_data_90.append(e_r_z_data_90)
    l_e_r_w_quat_data_90.append(e_r_w_data_90)
    l_d_r_x_quat_data_90.append(d_r_x_data_90)
    l_d_r_y_quat_data_90.append(d_r_y_data_90)
    l_d_r_z_quat_data_90.append(d_r_z_data_90)
    l_d_r_w_quat_data_90.append(d_r_w_data_90)
    
data_90['e_r_x']=l_e_r_x_quat_data_90
data_90['e_r_y']=l_e_r_y_quat_data_90
data_90['e_r_z']=l_e_r_z_quat_data_90
data_90['e_r_w']=l_e_r_w_quat_data_90
data_90['d_r_x']=l_d_r_x_quat_data_90
data_90['d_r_y']=l_d_r_y_quat_data_90
data_90['d_r_z']=l_d_r_z_quat_data_90
data_90['d_r_w']=l_d_r_w_quat_data_90



In [25]:
l_e_r_x_quat_data_180=[]
l_e_r_y_quat_data_180=[]
l_e_r_z_quat_data_180=[]
l_e_r_w_quat_data_180=[]
l_d_r_x_quat_data_180=[]
l_d_r_y_quat_data_180=[]
l_d_r_z_quat_data_180=[]
l_d_r_w_quat_data_180=[]
for index_m_180, row_m_180 in data_180.iterrows():
    e_r_x_euler_data_180,e_r_y_euler_data_180,e_r_z_euler_data_180=euler_from_quaternion(row_m_180[3],row_m_180[4],row_m_180[5],row_m_180[6])
    d_r_x_euler_data_180,d_r_y_euler_data_180,d_r_z_euler_data_180=euler_from_quaternion(row_m_180[10],row_m_180[11],row_m_180[12],row_m_180[13])
    
    e_r_z_euler_data_180=e_r_z_euler_data_180+np.pi
    d_r_z_euler_data_180=d_r_z_euler_data_180+np.pi
    
    e_r_x_data_180, e_r_y_data_180, e_r_z_data_180,e_r_w_data_180=get_quaternion_from_euler(e_r_x_euler_data_180, e_r_y_euler_data_180,e_r_z_euler_data_180)
    d_r_x_data_180, d_r_y_data_180, d_r_z_data_180,d_r_w_data_180=get_quaternion_from_euler(d_r_x_euler_data_180, d_r_y_euler_data_180,d_r_z_euler_data_180)
    
    l_e_r_x_quat_data_180.append(e_r_x_data_180)
    l_e_r_y_quat_data_180.append(e_r_y_data_180)
    l_e_r_z_quat_data_180.append(e_r_z_data_180)
    l_e_r_w_quat_data_180.append(e_r_w_data_180)
    l_d_r_x_quat_data_180.append(d_r_x_data_180)
    l_d_r_y_quat_data_180.append(d_r_y_data_180)
    l_d_r_z_quat_data_180.append(d_r_z_data_180)
    l_d_r_w_quat_data_180.append(d_r_w_data_180)
    
data_180['e_r_x']=l_e_r_x_quat_data_180
data_180['e_r_y']=l_e_r_y_quat_data_180
data_180['e_r_z']=l_e_r_z_quat_data_180
data_180['e_r_w']=l_e_r_w_quat_data_180
data_180['d_r_x']=l_d_r_x_quat_data_180
data_180['d_r_y']=l_d_r_y_quat_data_180
data_180['d_r_z']=l_d_r_z_quat_data_180
data_180['d_r_w']=l_d_r_w_quat_data_180



In [26]:
l_e_r_x_quat_data_270=[]
l_e_r_y_quat_data_270=[]
l_e_r_z_quat_data_270=[]
l_e_r_w_quat_data_270=[]
l_d_r_x_quat_data_270=[]
l_d_r_y_quat_data_270=[]
l_d_r_z_quat_data_270=[]
l_d_r_w_quat_data_270=[]
for index_m_270, row_m_270 in data_270.iterrows():
    e_r_x_euler_data_270,e_r_y_euler_data_270,e_r_z_euler_data_270=euler_from_quaternion(row_m_270[3],row_m_270[4],row_m_270[5],row_m_270[6])
    d_r_x_euler_data_270,d_r_y_euler_data_270,d_r_z_euler_data_270=euler_from_quaternion(row_m_270[10],row_m_270[11],row_m_270[12],row_m_270[13])
    
    e_r_z_euler_data_270=e_r_z_euler_data_270+np.pi*3/2
    d_r_z_euler_data_270=d_r_z_euler_data_270+np.pi*3/2
    
    e_r_x_data_270, e_r_y_data_270, e_r_z_data_270,e_r_w_data_270=get_quaternion_from_euler(e_r_x_euler_data_270, e_r_y_euler_data_270,e_r_z_euler_data_270)
    d_r_x_data_270, d_r_y_data_270, d_r_z_data_270,d_r_w_data_270=get_quaternion_from_euler(d_r_x_euler_data_270, d_r_y_euler_data_270,d_r_z_euler_data_270)
    
    l_e_r_x_quat_data_270.append(e_r_x_data_270)
    l_e_r_y_quat_data_270.append(e_r_y_data_270)
    l_e_r_z_quat_data_270.append(e_r_z_data_270)
    l_e_r_w_quat_data_270.append(e_r_w_data_270)
    l_d_r_x_quat_data_270.append(d_r_x_data_270)
    l_d_r_y_quat_data_270.append(d_r_y_data_270)
    l_d_r_z_quat_data_270.append(d_r_z_data_270)
    l_d_r_w_quat_data_270.append(d_r_w_data_270)
    
data_270['e_r_x']=l_e_r_x_quat_data_270
data_270['e_r_y']=l_e_r_y_quat_data_270
data_270['e_r_z']=l_e_r_z_quat_data_270
data_270['e_r_w']=l_e_r_w_quat_data_270
data_270['d_r_x']=l_d_r_x_quat_data_270
data_270['d_r_y']=l_d_r_y_quat_data_270
data_270['d_r_z']=l_d_r_z_quat_data_270
data_270['d_r_w']=l_d_r_w_quat_data_270



In [27]:
data_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.120,0.622,0.162,0.458186,0.466677,0.545427,0.524201,0.137,0.623,-0.193,...,-0.458301,-0.629763,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,0.120,0.622,0.162,0.458945,0.467419,0.544761,0.523569,0.137,0.623,-0.193,...,-0.458301,-0.629763,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,0.120,0.622,0.162,0.458945,0.467419,0.544761,0.523569,0.137,0.623,-0.193,...,-0.458301,-0.629763,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,0.120,0.622,0.162,0.458945,0.467419,0.544761,0.523569,0.137,0.623,-0.193,...,-0.458301,-0.629763,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,0.120,0.622,0.162,0.458943,0.467419,0.544069,0.524290,0.137,0.623,-0.193,...,-0.456736,-0.630483,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,0.331,0.652,0.156,0.446189,0.495686,0.513271,0.540151,0.172,0.615,-0.097,...,-0.514753,-0.527471,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,0.464,0.676,0.161,0.482954,0.577703,0.472317,0.458183,0.170,0.613,-0.097,...,-0.516832,-0.526704,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,0.525,0.682,0.166,0.472347,0.592602,0.457157,0.465531,0.168,0.611,-0.096,...,-0.513190,-0.528668,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,0.469,0.668,0.156,0.504165,0.549392,0.492549,0.448756,0.166,0.610,-0.095,...,-0.517548,-0.527425,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast


In [31]:
data_stop.to_csv('stop_90_quat.csv', index=False, header=True)
data_0.to_csv('0_90_quat.csv', index=False, header=True)
data_90.to_csv('90_90_quat.csv', index=False, header=True)
data_180.to_csv('180_90_quat.csv', index=False, header=True)
data_270.to_csv('270_90_quat.csv', index=False, header=True)

In [29]:
data_90.tail()

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
23993,-0.331,0.652,-0.156,0.495686,-0.446189,-0.540151,0.513271,-0.172,0.615,0.097,...,0.527471,-0.514753,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,-0.464,0.676,-0.161,0.577703,-0.482954,-0.458183,0.472317,-0.170,0.613,0.097,...,0.526704,-0.516832,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,-0.525,0.682,-0.166,0.592602,-0.472347,-0.465531,0.457157,-0.168,0.611,0.096,...,0.528668,-0.513190,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,-0.469,0.668,-0.156,0.549392,-0.504165,-0.448756,0.492549,-0.166,0.610,0.095,...,0.527425,-0.517548,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast
23997,-0.383,0.645,-0.149,0.511940,-0.514775,-0.440456,0.528132,-0.165,0.611,0.096,...,0.528071,-0.513972,0.049,2.274,0.271,0.019,0.017,-0.027,0.999,fast


In [30]:
data_0

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.162,0.622,0.120,0.006,-0.654,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.053,2.243,0.282,0.028,-0.031,-0.017,0.999,slow
1,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.053,2.242,0.286,0.030,-0.026,-0.016,0.999,slow
2,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.055,2.241,0.290,0.033,-0.028,-0.017,0.999,slow
3,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.015,0.193,0.623,0.137,...,0.769,0.121,-0.055,2.240,0.293,0.037,-0.029,-0.017,0.999,slow
4,-0.162,0.622,0.120,0.006,-0.655,-0.756,0.014,0.193,0.623,0.137,...,0.769,0.123,-0.057,2.239,0.297,0.039,-0.030,-0.017,0.999,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,-0.156,0.652,0.331,-0.035,0.666,0.745,0.019,0.097,0.615,0.172,...,0.737,0.009,0.115,2.276,0.250,0.013,0.019,-0.060,0.998,fast
23994,-0.161,0.676,0.464,0.067,-0.750,-0.658,0.010,0.097,0.613,0.170,...,0.738,0.007,0.114,2.275,0.251,0.008,0.023,-0.060,0.998,fast
23995,-0.166,0.682,0.525,-0.085,0.753,0.652,0.006,0.096,0.611,0.168,...,0.737,0.011,0.102,2.272,0.257,0.013,0.029,-0.054,0.998,fast
23996,-0.156,0.668,0.469,0.032,-0.745,-0.666,0.031,0.095,0.610,0.166,...,0.739,0.007,0.079,2.273,0.267,0.014,0.028,-0.043,0.999,fast
